In [1]:
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats

In [2]:
df_orders = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-tolstihin/ab_orders.csv' , sep = ',',parse_dates=['creation_time'])
df_orders

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19,"{75, 22, 53, 84}"
1,1256,2022-08-26 00:02:21,"{56, 76, 39}"
2,1257,2022-08-26 00:02:27,"{76, 34, 41, 38}"
3,1258,2022-08-26 00:02:56,"{74, 6}"
4,1259,2022-08-26 00:03:37,"{20, 45, 67, 26}"
...,...,...,...
4118,59422,2022-09-08 23:13:03,"{84, 8, 24}"
4119,59439,2022-09-08 23:18:05,"{9, 25, 75, 30, 6}"
4120,59464,2022-09-08 23:22:27,"{60, 41, 46}"
4121,59487,2022-09-08 23:29:06,"{9, 62, 77}"


In [3]:
df_users = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-tolstihin/ab_users_data.csv' , sep = ',',parse_dates=['time','date'])
df_users

,user_id,order_id,action,time,date,group
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0
1,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1
2,964,1257,create_order,2022-08-26 00:02:27,2022-08-26,0
3,966,1258,create_order,2022-08-26 00:02:56,2022-08-26,0
4,967,1259,create_order,2022-08-26 00:03:37,2022-08-26,1
...,...,...,...,...,...,...
4332,990,59422,create_order,2022-09-08 23:13:03,2022-09-08,1
4333,1418,59439,create_order,2022-09-08 23:18:05,2022-09-08,1
4334,1605,59464,create_order,2022-09-08 23:22:27,2022-09-08,0
4335,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0


In [4]:
df_prod = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-tolstihin/ab_products.csv' , sep = ',')
df_prod

,product_id,name,price
0,1,сахар,150.0
1,2,чай зеленый в пакетиках,50.0
2,3,вода негазированная,80.4
3,4,леденцы,45.5
4,5,кофе 3 в 1,15.0
...,...,...,...
82,83,вафли,55.0
83,84,мандарины,90.4
84,85,варенье,200.3
85,86,кофе холодный,70.3


In [5]:
df_users.action.unique()

array(['create_order', 'cancel_order'], dtype=object)

In [6]:
df_users.describe(include = 'datetime')

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,time,date
count,4337,4337
unique,4312,14
top,2022-09-07 19:59:53,2022-08-26 00:00:00
freq,2,1384
first,2022-08-26 00:00:19,2022-08-26 00:00:00
last,2022-09-08 23:41:24,2022-09-08 00:00:00


Исходя из описания предназначения умной системы, можно выдвинуть гипотезу о повышении в группе 1 частоты покупок (так как в данных всего 2 недели, то количество покупок), так как приложение стало эффективнее, и пользователям стало удобнее находить нужные товары.

In [7]:
df_users.groupby(['group'], as_index = False) \
    .agg({'user_id' : 'count'})

,group,user_id
0,0,1691
1,1,2646


In [8]:
df_users.groupby(['group'], as_index = False) \
    .agg({'user_id' : pd.Series.nunique})

,group,user_id
0,0,515
1,1,502


In [9]:
df_users.query("action == 'cancel_order'").groupby(['group'], as_index = False) \
    .agg({'user_id' : 'count'})

,group,user_id
0,0,82
1,1,132


In [10]:
df_users.query("action == 'cancel_order'").groupby(['group'], as_index = False) \
    .agg({'user_id' : pd.Series.nunique})

,group,user_id
0,0,76
1,1,113


In [11]:
#оставляем неотменненные заказы
df_users['flag'] = df_users.order_id.apply(lambda x: 'yes' if x in list(df_users[df_users.duplicated(['order_id'])].order_id) else 'no')

In [12]:
df_users.query("flag == 'no'")

,user_id,order_id,action,time,date,group,flag
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,no
1,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1,no
2,964,1257,create_order,2022-08-26 00:02:27,2022-08-26,0,no
4,967,1259,create_order,2022-08-26 00:03:37,2022-08-26,1,no
5,968,1261,create_order,2022-08-26 00:05:35,2022-08-26,0,no
...,...,...,...,...,...,...,...
4332,990,59422,create_order,2022-09-08 23:13:03,2022-09-08,1,no
4333,1418,59439,create_order,2022-09-08 23:18:05,2022-09-08,1,no
4334,1605,59464,create_order,2022-09-08 23:22:27,2022-09-08,0,no
4335,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0,no


In [13]:
df_users.query("flag == 'no'").groupby(['group'], as_index = False) \
    .agg({'user_id' : 'count'})

,group,user_id
0,0,1527
1,1,2382


In [14]:
df_users.query("flag == 'no'").groupby(['group'], as_index = False) \
    .agg({'user_id' : pd.Series.nunique})

,group,user_id
0,0,512
1,1,501


In [15]:
df_users.query("flag == 'no' and group == 0") \
    .groupby(['user_id'], as_index = False) \
    .agg({'date' : 'count'}) \
    .rename(columns={'date': 'count_buy'}) \
    .agg({'count_buy' : 'mean'})

count_buy    2.982422
dtype: float64

In [16]:
df_users.query("flag == 'no' and group == 1") \
    .groupby(['user_id'], as_index = False) \
    .agg({'date' : 'count'}) \
    .rename(columns={'date': 'count_buy'}) \
    .agg({'count_buy' : 'mean'})

count_buy    4.754491
dtype: float64

Нулевая гипотеза будет гласить о том, что в наших выборках никакого различия между средними значениями нет. Альтернативная будет говорить о том, что средние значимо различаются

In [17]:
stats.ttest_ind(df_users.query("flag == 'no' and group == 1") \
    .groupby(['user_id'], as_index = False) \
    .agg({'date' : 'count'}) \
    .rename(columns={'date': 'count_buy'}).count_buy, df_users.query("flag == 'no' and group == 0") \
    .groupby(['user_id'], as_index = False) \
    .agg({'date' : 'count'}) \
    .rename(columns={'date': 'count_buy'}).count_buy, equal_var=False)

Ttest_indResult(statistic=13.974169980221783, pvalue=1.2374101233656574e-40)

Можно сделать вывод о том, что среднее количество покупок на пользователя статистически значимо различаются и в экспериментальной группе это значение выше, что может говорить о положительном влиянии новой системы (при условии адекватности системы сплитования)

Также проверим гипотезу о влиянии новой системы на сумму покупок пользователей

In [18]:
df_prod

,product_id,name,price
0,1,сахар,150.0
1,2,чай зеленый в пакетиках,50.0
2,3,вода негазированная,80.4
3,4,леденцы,45.5
4,5,кофе 3 в 1,15.0
...,...,...,...
82,83,вафли,55.0
83,84,мандарины,90.4
84,85,варенье,200.3
85,86,кофе холодный,70.3


In [19]:
df_orders

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19,"{75, 22, 53, 84}"
1,1256,2022-08-26 00:02:21,"{56, 76, 39}"
2,1257,2022-08-26 00:02:27,"{76, 34, 41, 38}"
3,1258,2022-08-26 00:02:56,"{74, 6}"
4,1259,2022-08-26 00:03:37,"{20, 45, 67, 26}"
...,...,...,...
4118,59422,2022-09-08 23:13:03,"{84, 8, 24}"
4119,59439,2022-09-08 23:18:05,"{9, 25, 75, 30, 6}"
4120,59464,2022-09-08 23:22:27,"{60, 41, 46}"
4121,59487,2022-09-08 23:29:06,"{9, 62, 77}"


In [20]:
#считаем сумму заказа
def convert_sum(prod_ids):
    prod_ids = prod_ids.replace("{", "").replace("}", "").split(",")
    sum = 0
    for i in prod_ids:
        k = int(i)
        sum += float(df_prod.query("product_id == @k").price)
    return round(sum,3)

In [21]:
df_orders['order_sum'] = df_orders.product_ids.apply(convert_sum)

In [22]:
df_orders

,order_id,creation_time,product_ids,order_sum
0,1255,2022-08-26 00:00:19,"{75, 22, 53, 84}",408.7
1,1256,2022-08-26 00:02:21,"{56, 76, 39}",250.5
2,1257,2022-08-26 00:02:27,"{76, 34, 41, 38}",310.2
3,1258,2022-08-26 00:02:56,"{74, 6}",85.0
4,1259,2022-08-26 00:03:37,"{20, 45, 67, 26}",228.0
...,...,...,...,...
4118,59422,2022-09-08 23:13:03,"{84, 8, 24}",241.2
4119,59439,2022-09-08 23:18:05,"{9, 25, 75, 30, 6}",518.5
4120,59464,2022-09-08 23:22:27,"{60, 41, 46}",185.9
4121,59487,2022-09-08 23:29:06,"{9, 62, 77}",501.9


In [23]:
df_users

,user_id,order_id,action,time,date,group,flag
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,no
1,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1,no
2,964,1257,create_order,2022-08-26 00:02:27,2022-08-26,0,no
3,966,1258,create_order,2022-08-26 00:02:56,2022-08-26,0,yes
4,967,1259,create_order,2022-08-26 00:03:37,2022-08-26,1,no
...,...,...,...,...,...,...,...
4332,990,59422,create_order,2022-09-08 23:13:03,2022-09-08,1,no
4333,1418,59439,create_order,2022-09-08 23:18:05,2022-09-08,1,no
4334,1605,59464,create_order,2022-09-08 23:22:27,2022-09-08,0,no
4335,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0,no


In [24]:
df = pd.merge(
    df_users, df_orders[['order_id','order_sum']],
    left_on='order_id',
    right_on='order_id'
)

In [25]:
df

,user_id,order_id,action,time,date,group,flag,order_sum
0,964,1255,create_order,2022-08-26 00:00:19.000000,2022-08-26,0,no,408.7
1,965,1256,create_order,2022-08-26 00:02:21.000000,2022-08-26,1,no,250.5
2,964,1257,create_order,2022-08-26 00:02:27.000000,2022-08-26,0,no,310.2
3,966,1258,create_order,2022-08-26 00:02:56.000000,2022-08-26,0,yes,85.0
4,966,1258,cancel_order,2022-08-26 00:08:25.486419,2022-08-26,0,yes,85.0
...,...,...,...,...,...,...,...,...
4332,990,59422,create_order,2022-09-08 23:13:03.000000,2022-09-08,1,no,241.2
4333,1418,59439,create_order,2022-09-08 23:18:05.000000,2022-09-08,1,no,518.5
4334,1605,59464,create_order,2022-09-08 23:22:27.000000,2022-09-08,0,no,185.9
4335,1461,59487,create_order,2022-09-08 23:29:06.000000,2022-09-08,0,no,501.9


In [26]:
df.query("flag == 'no' and group == 0") \
    .groupby(['user_id'], as_index = False) \
    .agg({'order_sum' : 'sum'}) \
    .rename(columns={'order_sum': 'mean_sum'}) \
    .agg({'mean_sum' : 'mean'})

mean_sum    1139.562305
dtype: float64

In [27]:
df.query("flag == 'no' and group == 1") \
    .groupby(['user_id'], as_index = False) \
    .agg({'order_sum' : 'sum'}) \
    .rename(columns={'order_sum': 'mean_sum'}) \
    .agg({'mean_sum' : 'mean'})

mean_sum    1753.749301
dtype: float64

In [28]:
stats.ttest_ind(df.query("flag == 'no' and group == 0") \
    .groupby(['user_id'], as_index = False) \
    .agg({'order_sum' : 'sum'}).order_sum, df.query("flag == 'no' and group == 1") \
    .groupby(['user_id'], as_index = False) \
    .agg({'order_sum' : 'sum'}).order_sum, equal_var=False)

Ttest_indResult(statistic=-11.208629753614105, pvalue=1.572166429481266e-27)

Аналогично можно сделать вывод о принятии альтернативной гипотезы о статистически значимых результатах ведения новой системы.

И на заключительном этапе стоит проверить гипотезу о влиянии среднего количества отменненых заказов (при знакомстве с данными, визуально показалось, что пользователи стали больше отменять в группе 1)

In [29]:
df_users.query("flag == 'yes' and group == 0 and action == 'create_order'") \
    .groupby(['user_id'], as_index = False) \
    .agg({'date' : 'count'}) \
    .rename(columns={'date': 'count_buy'}) \
    .agg({'count_buy' : 'mean'})

count_buy    1.078947
dtype: float64

In [30]:
df_users.query("flag == 'yes' and group == 1 and action == 'create_order'") \
    .groupby(['user_id'], as_index = False) \
    .agg({'date' : 'count'}) \
    .rename(columns={'date': 'count_buy'}) \
    .agg({'count_buy' : 'mean'})

count_buy    1.168142
dtype: float64

In [31]:
stats.ttest_ind(df_users.query("flag == 'yes' and group == 1 and action == 'create_order'") \
    .groupby(['user_id'], as_index = False) \
    .agg({'date' : 'count'}) \
    .rename(columns={'date': 'count_buy'}).count_buy, df_users.query("flag == 'yes' and group == 0 and action == 'create_order'") \
    .groupby(['user_id'], as_index = False) \
    .agg({'date' : 'count'}) \
    .rename(columns={'date': 'count_buy'}).count_buy, equal_var=False)

Ttest_indResult(statistic=1.8296891493675829, pvalue=0.06888907857362961)

Т-тест показал, что не был достигнут нужный уровень значимости, и отклонить нулевую гипотезу мы не можем. 

В общем и целом, результаты A/B - тестирования показали, что можно, и даже нужно вводить новую систему, которая положительно влияет на суммы покупок пользователей и количество покупок пользователей. К сожалению, я не совсем успеваю выполнять задания в срок, хотя хотелось бы глубже ознакомиться с задачей.